In [36]:
import os
import pandas as pd
from preprocessing.data_cleaning import DataCleaner

df_raw_clima = pd.read_parquet('C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\1. Projetos Pessoais\\3. Previsão Público Cruzeiro\\data\\raw\\df_raw_clima.parquet',use_pandas_metadata=False, engine="pyarrow")
df_trusted_clima = pd.read_parquet('C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\1. Projetos Pessoais\\3. Previsão Público Cruzeiro\\data\\trusted\\df_trusted_clima.parquet',use_pandas_metadata=False, engine="pyarrow")

In [42]:
(df_trusted_clima.isna().sum() / df_trusted_clima.shape[0]) * 100

Data                                            0.000000
Hora UTC                                        0.000000
PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                0.092212
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)    0.092212
TEMPERATURA DO PONTO DE ORVALHO (°C)            0.092212
VENTO, VELOCIDADE HORARIA (m/s)                 0.093163
dtype: float64

In [46]:
df_trusted_clima[df_trusted_clima['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].isna()].groupby('Data').size()

Data
2020-09-03    10
2020-09-04    13
2020-09-05    13
2020-09-06    13
2020-09-07    14
2020-09-08    15
2020-09-09    11
2022-12-08     7
2022-12-20     1
dtype: int64

In [31]:
def tratar_nulos(df, coluna,valor_substituto):
    if coluna == "all":
        df = df.replace('None', valor_substituto)
        #df = df.fillna(valor_substituto)
        return df
    else:
        df[coluna] = df[coluna].replace('None', valor_substituto)
        #df[coluna] = df[coluna].fillna(valor_substituto)
        return df

In [3]:
cleaner = DataCleaner(df_raw_clima)

conversoes_ = {
        'Data': 'datetime64[ns]',
        'Hora UTC': 'string',    
        'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'float',
        'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'float',
        'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'float',
        'VENTO, VELOCIDADE HORARIA (m/s)': 'float',
    }

df_trusted_clima = (
    cleaner
    .tratar_divisor_milhar('TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)')
    .tratar_divisor_milhar('TEMPERATURA DO PONTO DE ORVALHO (°C)')
    .tratar_decimal('VENTO, VELOCIDADE HORARIA (m/s)')
    .tratar_decimal('PRECIPITAÇÃO TOTAL, HORÁRIO (mm)')
    #.tratar_nulos("all",0)
    #.ajustar_tipos_colunas(conversoes_)
    .get_df()
    )

In [35]:
x = tratar_nulos(df_trusted_clima,'all',None)

x.isna().sum()

Data                                             0
Hora UTC                                         0
PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                97
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)    97
TEMPERATURA DO PONTO DE ORVALHO (°C)            97
VENTO, VELOCIDADE HORARIA (m/s)                 98
dtype: int64

In [27]:
df_trusted_clima = df_trusted_clima.rename(columns={'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)':'precipitacao_total',
                                                    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)':'temp_media_bulbo_seco',
                                                    'TEMPERATURA DO PONTO DE ORVALHO (°C)':'temp_media_ponto_orvalho',
                                                    'VENTO, VELOCIDADE HORARIA (m/s)':'velocidade_media_vento',})

In [28]:
teste = df_trusted_clima.groupby(['Data']).agg({'precipitacao_total': 'sum',
                                                'temp_media_bulbo_seco': 'mean',
                                                'temp_media_ponto_orvalho': 'mean',
                                                'velocidade_media_vento': 'mean'}).reset_index()

In [30]:
def feature_lag_clima(df, coluna):
    df[f'{coluna}_lag1'] = df[coluna].shift(1)
    df[f'{coluna}_lag2'] = df[coluna].shift(2)
    df[f'{coluna}_lag3'] = df[coluna].shift(3)
    df[f'{coluna}_lag4'] = df[coluna].shift(4)
    df[f'{coluna}_lag5'] = df[coluna].shift(5)
    df[f'{coluna}_lag6'] = df[coluna].shift(6)
    df[f'{coluna}_lag7'] = df[coluna].shift(7)
    return df

In [31]:
feature_lag_clima(teste, 'precipitacao_total')

,Data,precipitacao_total,temp_media_bulbo_seco,temp_media_ponto_orvalho,velocidade_media_vento,precipitacao_total_lag1,precipitacao_total_lag2,precipitacao_total_lag3,precipitacao_total_lag4,precipitacao_total_lag5,precipitacao_total_lag6,precipitacao_total_lag7
0,2019-01-01,28.2,21.833333,18.566667,0.987500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-02,0.0,24.091667,18.058333,1.100000,28.2,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-03,0.0,24.341667,18.537500,1.037500,0.0,28.2,NaN,NaN,NaN,NaN,NaN
3,2019-01-04,10.0,22.337500,18.008333,1.358333,0.0,0.0,28.2,NaN,NaN,NaN,NaN
4,2019-01-05,0.0,23.550000,17.066667,1.000000,10.0,0.0,0.0,28.2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2022-12-27,7.2,21.741667,18.025000,1.058333,13.6,1.4,0.0,0.0,0.0,53.2,19.2
1457,2022-12-28,0.0,23.316667,17.933333,1.383333,7.2,13.6,1.4,0.0,0.0,0.0,53.2
1458,2022-12-29,1.8,22.708333,18.320833,1.116667,0.0,7.2,13.6,1.4,0.0,0.0,0.0
1459,2022-12-30,21.0,22.720833,17.629167,1.187500,1.8,0.0,7.2,13.6,1.4,0.0,0.0
